In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from torchtext import datasets
import pytorch_lightning as pl

In [2]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root='../data/')

In [4]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [29]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
BATCH_SIZE = 32

In [30]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of testing examples: 25000


In [31]:
print(vars(train_data.examples[0]))

{'text': ['the', 'more', 'i', 'think', 'about', 'it', ',', 'there', 'was', 'nothing', 'redeeming', 'about', 'this', '<', 'br', '/><br', '/>movie', '.', 'i', 'saw', 'it', '9', 'months', 'ago', ',', 'so', 'my', 'memory', 'might', 'have', 'made', 'it', '<', 'br', '/><br', '/>worse', 'than', 'it', 'was', ',', 'but', 'i', 'do', 'know', 'it', 'was', 'at', 'least', 'as', 'bad', 'as', 'a', '4', 'out', 'of', '<', 'br', '/><br', '/>10', '.', '<', 'br', '/><br', '/>after', 'seeing', 'the', 'movie', ',', 'i', 'met', 'the', 'director', '.', 'he', 'seemed', 'so', 'clueless', '<', 'br', '/><br', '/>as', 'to', 'what', 'he', 'was', 'doing', 'or', 'what', 'he', 'had', 'done', ',', 'and', 'as', 'far', 'as', 'i', '<', 'br', '/><br', '/>could', 'tell', ',', 'he', 'did', "n't", 'care', 'for', 'the', 'film', 'either', '.', 'even', 'he', 'agreed', 'that', 'he', '<', 'br', '/><br', "/>didn't", 'really', 'know', 'what', 'he', 'was', 'doing', ',', 'and', 'he', 'was', 'forced', 'to', 'do', '<', 'br', '/><br', '/>

In [32]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [33]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [34]:
print(TEXT.vocab.freqs.most_common(100))

[('the', 202076), (',', 191932), ('.', 166030), ('and', 109303), ('a', 109087), ('of', 100707), ('to', 93450), ('is', 76015), ('in', 61243), ('I', 54556), ('it', 53404), ('that', 49053), ('"', 43853), ("'s", 43102), ('this', 42382), ('-', 36825), ('/><br', 35520), ('was', 35174), ('as', 30150), ('with', 29898), ('movie', 29885), ('for', 29404), ('film', 27053), ('The', 26178), ('but', 24534), ('(', 23177), ("n't", 23033), (')', 23001), ('on', 22909), ('you', 21234), ('are', 20803), ('not', 20101), ('have', 19708), ('his', 19409), ('be', 18555), ('he', 17231), ('one', 16747), ('by', 15252), ('at', 15207), ('!', 15008), ('all', 14998), ('an', 14638), ('who', 14519), ('they', 13771), ('from', 13683), ('like', 13674), ('so', 12337), ('or', 12000), ('her', 11974), ('about', 11902), ('out', 11769), ('has', 11751), ('It', 11726), ("'", 11718), ('just', 11656), ('do', 11097), ('?', 10459), ('some', 10122), ('good', 10025), ('more', 9664), ('would', 9292), ('very', 9283), ('up', 9058), ('what',

In [35]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)



In [107]:
it = next(iter(train_iterator))
text, ln = it.text
# label, ln = it.label
# print([TEXT.vocab.itos[token] for token in text.tolist()])
print([TEXT.vocab.itos[token] for token in it.text[0][0].tolist()])
print('label : ',LABEL.vocab.itos[int(it.label[0].item())])

# it.label[0].item()

# LABEL.vocab.itos

['Vertigo', 'Being', 'This', '"', 'What', 'Terrible', 'There', 'the', 'This', 'A', 'A', 'Just', 'In', 'This', 'A', 'This', 'Not', 'Watching', 'This', 'I', 'I', "'", 'If', 'Not', '"', '"', 'An', 'Do', 'After', 'I', 'This', 'Sad']
label :  pos


In [126]:
tr = train_data[0]
tr.text
# text, ln = it.text
# # label, ln = it.label
print([TEXT.vocab.itos[token] for token in tr.text])
# print([TEXT.vocab.itos[token] for token in it.text[0][0].tolist()])
# print('label : ',LABEL.vocab.itos[int(it.label[0].item())])

# it.label[0].item()

# LABEL.vocab.itos

TypeError: list indices must be integers or slices, not str

In [75]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [76]:
class SentimentNetwork(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

    
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)    

class TaskIMDB(pl.LightningModule):
    def __init__(self, model, optimizers, criterion, accuracy):
        super().__init__()
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.accuracy = accuracy
        
    def shared_step(self, batch, batch_idx):
        text, text_lengths = batch.text
        predictions = self.model(text, text_lengths).squeeze(1)
#         predictions = self.model(batch.text).squeeze(1)
        loss = self.criterion(predictions, batch.label)
        acc = self.accuracy(predictions, batch.label)
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch, batch_idx)
        result = pl.TrainResult(loss)
        result.log_dict({'trn_loss': loss, 'trn_acc':acc})
        return result
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch, batch_idx)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log_dict({'val_loss': loss, 'val_acc': acc})
        return result
    
    def configure_optimizers(self):
        return self.optimizer
    
  

In [80]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


In [81]:
# model = SentimentNetwork(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)


PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = SentimentLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)




In [82]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [83]:
from pytorch_lightning import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger('logs/imdb')
task = TaskIMDB(model, optimizer, criterion, binary_accuracy)
trainer = pl.Trainer(gpus=1, logger=tb_logger)
trainer.fit(task, train_iterator, valid_iterator)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | SentimentLSTM     | 4 M   
1 | criterion | BCEWithLogitsLoss | 0     


Saving latest checkpoint..


1

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [31]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
N_EPOCHS = 5

best_valid_loss = float('inf')

model = model.to(device)
criterion = criterion.to(device)


for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 49.68%
	 Val. Loss: 0.698 |  Val. Acc: 50.72%
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 50.20%
	 Val. Loss: 0.698 |  Val. Acc: 49.22%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 50.21%
	 Val. Loss: 0.698 |  Val. Acc: 48.73%
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 50.39%
	 Val. Loss: 0.698 |  Val. Acc: 48.97%
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.693 | Train Acc: 49.81%
	 Val. Loss: 0.698 |  Val. Acc: 48.98%


In [35]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select